In [24]:
import numpy as np
import psycopg2
import matplotlib.pyplot as plt

# Connect to DB containing the exported tables using import-osm
# Based on martinelli's work
connection = psycopg2.connect(
    database="noise",
    user="noise",
    password="noise",
    host="localhost",
    port=62341 # Change to the port postgres is accessible from
)
cursor = connection.cursor()

## Query
This query gets all geometries within x meters of certain long, lat of a certain type

In [25]:
q = """
SELECT
    id,
    ST_Distance(ST_Transform(ST_SetSrid(ST_Point({0}, {1}), 4326), 3857), geometry) * cosd({1}) dist
FROM {2}
WHERE
    ST_DWithin(ST_Transform(ST_SetSrid(ST_Point({0}, {1}), 4326), 3857), geometry, {3} / cosd({1}))
ORDER BY dist
LIMIT {4}
"""

## Config
* tables: the table names containing the geometries. Mapping to the noise produced by each geometry
* limits: How many rows to retreive from each table for measuring noise

In [26]:
tables = {
    "osm_light_railway": 80,
    "osm_secondary_road": 87,
    "osm_trunk": 100,
    "osm_motorway": 110,
    "osm_tertiary_road": 80,
    "osm_primary_road": 95,
    "osm_railway": 85,
    "osm_retail_zone": 90,
    "osm_sport": 70,
    "osm_leisure": 65,
    "osm_party": 80,
    "osm_shop": 65,
    "osm_industrial_zone": 90,
    "osm_food": 65,
}
limits = {
    "osm_light_railway": 1,
    "osm_secondary_road": 1,
    "osm_trunk": 1,
    "osm_motorway": 1,
    "osm_tertiary_road": 1,
    "osm_primary_road": 1,
    "osm_railway": 1,
    "osm_retail_zone": 10,
    "osm_sport": 10,
    "osm_leisure": 10,
    "osm_party": 10,
    "osm_shop": 10,
    "osm_industrial_zone": 10,
    "osm_food": 10,
}

## Endpoint

A function that calculates noise given long, lat

In [27]:
def noise(long, lat, max_dist=1200):
    """Noise report for certain long, lat
    
    Args:
        long: Longitude
        lat: Latitude
        max_dist: Maximum distance to look for geometries for
        
    Returns:
        dict containing the report of the structure
        {'osm_secondary_road': 166,
         'osm_tertiary_road': 355,
         'osm_primary_road': 234,
         'osm_railway': 184,
         'osm_retail_zone': 272,
         'osm_party': 234,
         'osm_shop': 163,
         'osm_industrial_zone': 272,
         'osm_food': 271,
         'noise': 49.9986629248497}
    """
    dist  = [] # Distances
    power = [] # Noise Power
    info  = {} # Nearest of each type
    
    # get nearest x geometries of each type
    for table in tables:
        cursor.execute(q.format(long, lat, table, max_dist, limits[table]))
        record = cursor.fetchall()
        for r in record:
            dist.append(round(r[1]))
            power.append(tables[table])
            info[table] = info.get(table, round(r[1]))

    # Noise calculation formula (simple noise propagations)
    dist = np.array(dist).clip(0.01)
    power = np.array(power)
    info["noise"] = np.log10((10**(power / 10 - 2 * abs(np.log10(dist)) - dist / 3000)).sum()) * 10
    return info

In [28]:
%%time
noise(8.444360, 50.082364)

CPU times: user 2.09 ms, sys: 2.43 ms, total: 4.52 ms
Wall time: 51.5 ms


{'osm_secondary_road': 166,
 'osm_tertiary_road': 355,
 'osm_primary_road': 234,
 'osm_railway': 184,
 'osm_retail_zone': 272,
 'osm_party': 234,
 'osm_shop': 163,
 'osm_industrial_zone': 272,
 'osm_food': 271,
 'noise': 49.9986629248497}